In [1]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import unidecode
import re


In [2]:
# Código para fazer a leitura de um arquivo json direto

# df = open('DODF 002 04-01-2022.json')

# data = json.load(df)

# data["json"]

In [3]:
def parse_html_text(text):
    if text is not None:
        return unidecode.unidecode(BeautifulSoup(text, 'html.parser').get_text())
    
    return text

In [4]:
page = requests.get("http://164.41.76.30/dodf_watcher/")
html = page.content.decode("utf-8")

soup = BeautifulSoup(html, 'html.parser')

url_paths = soup.findAll('a')
url_paths = [l.get_text() for l in url_paths]
url_paths = [l.replace("\n", "%0A") for l in url_paths if "DODF" in l]

documentos = {"dodf": [],
              "link_jornal": [],
              "secao": [],
              "orgao": [],
              "documento": []}

secao = "Seção III"

In [5]:
for dodf in url_paths[:10]:
    r = requests.get("http://164.41.76.30/dodf_watcher/" + dodf)
    r_json = r.json()["json"]
    
    if secao in r_json["INFO"]:
        for orgao in r_json["INFO"][secao]:
            for doc in r_json["INFO"][secao][orgao]["documentos"]:
                documentos["dodf"].append(dodf)
                documentos["link_jornal"].append(r_json["linkJornal"])
                documentos["secao"].append(secao)
                documentos["orgao"].append(orgao)
                documentos["documento"].append(r_json["INFO"][secao][orgao]["documentos"][doc])
    
    print(dodf + " feito.")



DODF 002 04-01-2022.json feito.
DODF 003 05-01-2022.json feito.
DODF 004 06-01-2022.json feito.
DODF 005 07-01-2022.json feito.
DODF 006 10-01-2022.json feito.
DODF 007 11-01-2022.json feito.
DODF 008 12-01-2022.json feito.
DODF 009 13-01-2022.json feito.
DODF 010 14-01-2022.json feito.
DODF 011 17-01-2022.json feito.


In [6]:
documentos_mod = documentos.copy()
doc_keys = []

In [7]:
for doc in documentos_mod["documento"]:
    doc_keys += list(doc.keys())
    
doc_keys = list(set(doc_keys))

for k in doc_keys:
    documentos_mod[k] = []

for doc in documentos_mod["documento"]:
    for k in doc_keys:
        if k in doc.keys():
            documentos_mod[k].append(doc[k])
        else:
            documentos_mod[k].append(None)

In [8]:
documentos_mod.pop("documento")

documentos_df = pd.DataFrame(documentos_mod)
documentos_df["dodf"] = documentos_df["dodf"].str.replace(".json", "")
documentos_df = documentos_df.drop(columns=["coDemandante", "situacao", "coMateria", "layout", "nuOrdem", "regraSituacao", "preambulo", "noDemandante"])

documentos_df["texto"] = documentos_df["texto"].apply(lambda text: parse_html_text(text))



C:\Users\ianfi\AppData\Local\Temp\ipykernel_25400\97987316.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  documentos_df["dodf"] = documentos_df["dodf"].str.replace(".json", "")


In [9]:
documentos_df

,dodf,link_jornal,secao,orgao,tipo,titulo,texto
0,DODF 002 04-01-2022,do.df.gov.br/220104NO002,Seção III,Poder Legislativo,Extrato,EXTRATO DE TERMO DE CREDENCIAMENTO,Processo: 00001-00023871/2020-25. Contrato no ...
1,DODF 002 04-01-2022,do.df.gov.br/220104NO002,Seção III,Secretaria de Estado de Governo,Extrato,EXTRATO DO CONTRATO DE PRESTAÇÃO DE SERVIÇOS N...,NOS TERMOS DO PADRAO No 05/2002Processo: 00133...
2,DODF 002 04-01-2022,do.df.gov.br/220104NO002,Seção III,Secretaria de Estado de Governo,Extrato,EXTRATO DE NOTA DE EMPENHO Nº 2021NE00035,Processo: 04022-00000121/2021-41. Partes: DIST...
3,DODF 002 04-01-2022,do.df.gov.br/220104NO002,Seção III,Secretaria de Estado de Economia,Extrato,EXTRATO DO TERCEIRO TERMO ADITIVO AO CONTRATO ...,Processo: 00040-00058335/2017-24. A SECRETARIA...
4,DODF 002 04-01-2022,do.df.gov.br/220104NO002,Seção III,Secretaria de Estado de Economia,Extrato,EXTRATO DO PRIMEIRO TERMO ADITIVO AO CONTRATO ...,Processo: 00040-00008865/2021-16. A SECRETARIA...
...,...,...,...,...,...,...,...
1422,DODF 011 17-01-2022,do.df.gov.br/220117NO011,Seção III,Secretaria de Estado de Meio Ambiente,Aviso,AVISO DE CHAMAMENTO PÚBLICO Nº 01/2022,"A FUNDACAO JARDIM ZOOLOGICO DE BRASILIA, torna..."
1423,DODF 011 17-01-2022,do.df.gov.br/220117NO011,Seção III,Secretaria de Estado de Trabalho,Extrato,EXTRATO DO PRIMEIRO TERMO ADITIVO AO CONTRATO ...,Processo: 04012-00001806/2021-42. DAS PARTES: ...
1424,DODF 011 17-01-2022,do.df.gov.br/220117NO011,Seção III,Ineditorial,Ineditorial,CHAMAMENTO Nº 025/2022,PROCESSO: 04024-00000087/2022-30O instituto do...
1425,DODF 011 17-01-2022,do.df.gov.br/220117NO011,Seção III,Ineditorial,Ineditorial,AVISO DE REQUERIMENTO DE RENOVAÇÃO DA LICENÇA ...,Torna publico que esta requerendo do Instituto...


In [10]:
def extract_acts_suspensao(dataframe):
    regex = r'(?:xxbcet\s+)?(?:AVISO\s+D[EO]\s+SUSPENS[AÃ]O\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISO\s+D[EO]\s+SUSPENS[AÃ]O)'

    lista = []
    for i in range(len(dataframe)):
        if re.match(regex, dataframe['titulo'][i]):
            lista.append(i)

    return dataframe.iloc[lista].reset_index(drop=True)

In [11]:
def extract_acts_anulacao_revogacao(dataframe):
    regex = r'(?:xxbcet\s+)?(?:AVISO\s+D[EO]\s+REVOGA[CÇ][AÃ]O\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISO\s+D[EO]\s+REVOGA[CÇ][AÃ]O|AVISO\s+D[EO]\s+ANULA[CÇ][AÃ]O\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISO\s+D[EO]\s+ANULA[CÇ][AÃ]O)'

    lista = []
    for i in range(len(dataframe)):
        if re.match(regex, dataframe['titulo'][i]):
            lista.append(i)

    return dataframe.iloc[lista].reset_index(drop=True)

In [12]:
df_suspensao = extract_acts_suspensao(documentos_df)
df_anulacao_revogacao = extract_acts_anulacao_revogacao(documentos_df)

In [13]:
df_suspensao

,dodf,link_jornal,secao,orgao,tipo,titulo,texto
0,DODF 003 05-01-2022,do.df.gov.br/220105NO003,Seção III,Secretaria de Estado de Trabalho,Aviso,AVISO DE SUSPENSÃO,PREGAO ELETRONICO No 26/2021 - UASG 926210Proc...
1,DODF 008 12-01-2022,do.df.gov.br/220112NO008,Seção III,Secretaria de Estado de Saúde,Aviso,AVISO DE SUSPENSÃO,PREGAO ELETRONICO POR SRP No 444/2021 - UASG 9...
2,DODF 008 12-01-2022,do.df.gov.br/220112NO008,Seção III,Secretaria de Estado de Segurança Pública,Aviso,AVISO DE SUSPENSÃO DE LICITAÇÃO,PREGAO ELETRONICO No 37/2021Processo: 00010.00...
3,DODF 010 14-01-2022,do.df.gov.br/220114NO010,Seção III,Secretaria de Estado de Educação,Aviso,AVISO DE SUSPENSÃO,PREGAO ELETRONICO POR SRP No 01/2022 (UASG 450...


In [14]:
df_anulacao_revogacao

,dodf,link_jornal,secao,orgao,tipo,titulo,texto
0,DODF 005 07-01-2022,do.df.gov.br/220107NO005,Seção III,Secretaria de Estado de Segurança Pública,Aviso,AVISO DE ANULAÇÃO DE LICITAÇÃO,PREGAO ELETRONICO No 32/2021Processo: 00050-00...
1,DODF 005 07-01-2022,do.df.gov.br/220107NO005,Seção III,Secretaria de Estado de Obras e Infraestrutura,Aviso,AVISO DE REVOGAÇÃO,Comunicamos aos interessados na Concorrencia n...
2,DODF 011 17-01-2022,do.df.gov.br/220117NO011,Seção III,Secretaria de Estado de Esporte e Lazer,Aviso,AVISO DE REVOGAÇÃO EDITAL CHAMAMENTO PÚBLICO N...,A SECRETARIA DE ESTADO DE ESPORTE E LAZER DO D...
